# DD142X

### Prerequisites
* Python v3.7
* Tensorflow v1.4.2 
* Installera massa skit med pip för att få det att fungera
* Jag ändrade i train_image_classifier.py för att träna på CPU

Det kan funka med andra versioner, men inte Tensorflow 2.

### Ladda in directories

In [6]:
# Where the pre-trained InceptionV3 checkpoint is saved to.
%env PRETRAINED_CHECKPOINT_DIR=/tmp/checkpoints

# Where the training (fine-tuned) checkpoint and logs will be saved to.
%env TRAIN_DIR=/tmp/cancer-models/inception_v3

# Where the dataset is saved to.
%env DATASET_DIR=/tmp/data/flowers

env: PRETRAINED_CHECKPOINT_DIR=/tmp/checkpoints
env: TRAIN_DIR=/tmp/cancer-models/inception_v3
env: DATASET_DIR=/tmp/data/flowers


In [7]:
!python3 download_and_convert_data.py \
    --dataset_name=flowers \
    --dataset_dir="${DATASET_DIR}"


W0504 11:57:00.058064 4722865600 module_wrapper.py:139] From /Users/johan/Dev/kexet/models/research/slim/datasets/download_and_convert_flowers.py:183: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.


W0504 11:57:00.058404 4722865600 module_wrapper.py:139] From /Users/johan/Dev/kexet/models/research/slim/datasets/download_and_convert_flowers.py:184: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

>> Downloading flower_photos.tgz 100.0%
Successfully downloaded flower_photos.tgz 228813984 bytes.

W0504 11:57:22.499127 4722865600 module_wrapper.py:139] From /Users/johan/Dev/kexet/models/research/slim/datasets/download_and_convert_flowers.py:57: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.


W0504 11:57:22.500859 4722865600 module_wrapper.py:139] From /Users/johan/Dev/kexet/models/research/slim/datasets/download_and_convert_flowers.py:124: The name tf.Session is deprecated. Please use

### Transfer learning
* Ladda vikterna från den generella färdigtränade modellen, men exkludera sista lagret med "checkpoint_exclude_scopes". 
* Träna om vikterna i detta lager med "trainable_scopes".

In [4]:
# Fine-tune only the new layers for 1000 steps.
!python3 train_image_classifier.py \
  --train_dir=${TRAIN_DIR} \
  --dataset_name=flowers \
  --dataset_split_name=train \
  --dataset_dir=${DATASET_DIR} \
  --model_name=inception_v3 \
  --checkpoint_path=${PRETRAINED_CHECKPOINT_DIR}/inception_v3.ckpt \
  --checkpoint_exclude_scopes=InceptionV3/Logits,InceptionV3/AuxLogits \
  --trainable_scopes=InceptionV3/Logits,InceptionV3/AuxLogits \
  --max_number_of_steps=1000 \
  --batch_size=32 \
  --learning_rate=0.01 \
  --learning_rate_decay_type=fixed \
  --save_interval_secs=60 \
  --save_summaries_secs=60 \
  --log_every_n_steps=100 \
  --optimizer=rmsprop \
  --weight_decay=0.00004



W0504 11:54:03.986433 4411241920 module_wrapper.py:139] From train_image_classifier.py:417: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0504 11:54:03.986618 4411241920 module_wrapper.py:139] From train_image_classifier.py:417: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

Instructions for updating:
Please switch to tf.train.create_global_step
W0504 11:54:03.991581 4411241920 deprecation.py:323] From train_image_classifier.py:431: create_global_step (from tensorflow.contrib.framework.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.create_global_step

W0504 11:54:04.002713 4411241920 module_wrapper.py:139] From /Users/johan/Dev/kexet/models/research/slim/datasets/flowers.py:74: The name tf.FixedLenFeature is deprecated. Please use tf.io.FixedLenFeature instead.


W0504 11:54:04.003545 4411241920 module_

In [ ]:
# Run evaluation.
python eval_image_classifier.py \
  --checkpoint_path=${TRAIN_DIR} \
  --eval_dir=${TRAIN_DIR} \
  --dataset_name=flowers \
  --dataset_split_name=validation \
  --dataset_dir=${DATASET_DIR} \
  --model_name=inception_v3

In [ ]:
import json
import os

path_to_descriptions = '/kdlskdlsd'

final_path = '/tmp/data/skincancer'

for root, dirs, files in os.walk(path_to_desriptions):
    for name in files:
        src = root + '/' + name
        with open(src) as data_file:
            jsonList = json.load(data_file)
            
            disease = jsonList['meta']['clinical']['diagnosis']
            image_src_path = '../' + name + '.jpeg'
            disease_dir = final_path + '/' + disease
            if not os.path.exists(disease_dir):
                os.makedirs(disease_dir)
            
            data_file.close()
            shutil.copy(image_src_path, disease_dir)
            
print("Finished!")